In [1]:
import time
import asyncio
import aiohttp
import requests
import threading
import pandas as pd

In [2]:
cols = ['id', 'name',  'age_ratings', 'slug', 'aggregated_rating', 'aggregated_rating_count', 'category', 'collection', 
        'alternative_names', 'external_games', 'first_release_date', 'game_engines', 'game_modes', 'genres', 
        'involved_companies', 'keywords', 'platforms', 'rating', 'player_perspectives', 'dlcs', 'franchise',
        'rating_count', 'release_dates', 'similar_games', 'tags', 'parent_game', 'status',
        'total_rating', 'total_rating_count', 'updated_at', 'websites']

In [3]:
client_id = 'x0rncah74zueqqov31fhkwuzlqv27a'
access_token = 'Bearer lhty0tvjq3el3s4rufgiw5vufmjk31'

In [4]:
async def create_games_dataset(floor, ceiling, endpoint='games'):
    # нужно для того, чтобы авторизироваться
    headers = {'Client-ID': client_id, 
               'Authorization': access_token}
    async with aiohttp.ClientSession() as session:
        all_games = pd.DataFrame(columns=cols)
        for i in range(floor, ceiling):
            # endpoint и id можно изменять для того, чтобы получить разные наборы
            body = 'fields *; where id = ' + str(i) + ';'
            async with session.post('https://api.igdb.com/v4/' + endpoint + '/', headers=headers, data=body) as resp:
                game = await resp.json()
                try: # если вышла такая ошибка, значит слишком много запросов в одну секунду
                    # на тот случай, если id нет в базе данных
                    if len(game) > 0:
                        all_games.loc[len(all_games.index)] = game[0]
                except KeyError: # поэтому, ждем 1 секунду и делаем новый запрос
                    time.sleep(3)
                    async with session.post('https://api.igdb.com/v4/' + endpoint + '/', headers=headers, data=body) as resp:
                        game = await resp.json()
                        if len(game) > 0:
                            all_games.loc[len(all_games.index)] = game[0]
        return all_games

In [13]:
start_time = time.time()
all_fields_one = await create_games_dataset(120000, 140000)
print("--- %s seconds ---" % (time.time() - start_time))

--- 13985.45098233223 seconds ---


In [15]:
all_fields_one.to_csv('C:/Users/ASDW/Python/Projects/Recosys 2.0/datasets/games_dataset/new_games_dataset.csv', mode = 'a', index=False)

In [14]:
all_fields_one.head()

,id,name,age_ratings,slug,aggregated_rating,aggregated_rating_count,category,collection,alternative_names,external_games,...,rating_count,release_dates,similar_games,tags,parent_game,status,total_rating,total_rating_count,updated_at,websites
0,120000,Click Commander,NaN,click-commander,NaN,NaN,0,NaN,NaN,"[1725670, 1985966]",...,NaN,NaN,"[18623, 31515, 33603, 36962, 51577, 55888, 658...","[268435469, 268435471, 268435488]",NaN,4,NaN,NaN,1629801261,[111191]
1,120001,0Gravity,NaN,0gravity,NaN,NaN,0,NaN,NaN,"[1725674, 1987147]",...,NaN,[234006],"[32902, 37419, 40524, 76263, 96217, 105233, 11...","[1, 268435488]",NaN,4,NaN,NaN,1627305686,[110508]
2,120002,The Itch,NaN,the-itch,NaN,NaN,0,NaN,NaN,"[1725675, 1988843]",...,NaN,[192536],"[19150, 28070, 37419, 55173, 55190, 96217, 106...","[268435464, 268435487, 268435488, 536871029, 5...",NaN,NaN,NaN,NaN,1627290043,"[111370, 111371]"
3,120003,Un Pas Fragile,NaN,un-pas-fragile,NaN,NaN,0,NaN,NaN,"[1725676, 1953478]",...,0,[187668],"[28277, 37419, 55282, 75240, 75948, 87514, 879...","[268435487, 268435488]",NaN,NaN,70.0,0,1633079859,"[111247, 111248, 131796]"
4,120004,Eat this!,NaN,eat-this,NaN,NaN,0,NaN,NaN,"[1725677, 1994578]",...,NaN,[283202],"[28277, 37419, 51577, 55888, 65827, 107196, 11...","[268435466, 268435469, 268435470, 268435488]",NaN,NaN,NaN,NaN,1630310280,[110352]
